# MDL resistance

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

In [ ]:
runs_meta_data = catalog.load('runs_meta_data')
runs_meta_data.dropna(how='any', axis=1, inplace=True)

In [ ]:
runs_meta_data.head()

In [ ]:
runs_meta_data.columns

In [ ]:
reference_speeds = runs_meta_data.groupby(by='test_type').get_group('reference speed')

In [ ]:
df_mean = pd.DataFrame()
df_std = pd.DataFrame()


for id, row in reference_speeds.iterrows():
    
    try:
        df_ = catalog.load(f'{id}.data_ek_smooth')
    except:
        continue
    
    s_mean = df_.mean()
    s_mean.name = id
    df_mean = df_mean.append(s_mean)
    
    s_std = df_.std()
    s_std.name = id
    df_std = df_std.append(s_std)
    
    

In [ ]:
df_upper = df_mean.copy()
df_upper['thrust']+=3*df_std['thrust']
df_lower = df_mean.copy()
df_lower['thrust']-=3*df_std['thrust']
df_mean.sort_values(by='u', inplace=True)
df_upper.sort_values(by='u', inplace=True)
df_lower.sort_values(by='u', inplace=True)

In [ ]:
fig,ax=plt.subplots()

df_mean.plot(x='u', y='thrust', style='.-', ax=ax)
df_lower.plot(x='u', y='thrust', style='-', label='lower', ax=ax)
df_upper.plot(x='u', y='thrust', style='-', label='upper', ax=ax)
ax.legend();


In [ ]:
df_mean